In [7]:
pip install torch transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00


In [8]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
sentence = "This was one of the best movies I have ever seen."
inputs = tokenizer(sentence, return_tensors = 'pt')#The tokenizer takes the input sentence and converts it into a sequence of tokens that can be fed into a neural network. 
print(inputs)
#Each number in this tensor corresponds to a specific token in the vocabulary of the distilbert-base-uncased model. For example, 101 corresponds to the [CLS] token, which is a special token used to indicate the beginning of a sequence. 2023 corresponds to the token for the word "this", 2001 corresponds to the token for "was", and so on

{'input_ids': tensor([[ 101, 2023, 2001, 2028, 1997, 1996, 2190, 5691, 1045, 2031, 2412, 2464,
         1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [9]:
#from transformers import pipeline
#unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
#unmasker("Hello I'm a [MASK] model.")


In [10]:
! wget https://tubcloud.tu-berlin.de/s/GfEoq4r8Sgb2727/download/distilbert.pt

--2023-05-07 15:37:22--  https://tubcloud.tu-berlin.de/s/GfEoq4r8Sgb2727/download/distilbert.pt
Resolving tubcloud.tu-berlin.de (tubcloud.tu-berlin.de)... 141.23.34.165
Connecting to tubcloud.tu-berlin.de (tubcloud.tu-berlin.de)|141.23.34.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267854575 (255M) [application/octet-stream]
Saving to: ‘distilbert.pt’

distilbert.pt       100%[===================>] 255.45M  4.20MB/s    in 63s     

2023-05-07 15:38:26 (4.08 MB/s) - ‘distilbert.pt’ saved [267854575/267854575]



In [3]:
from torch import nn 
import torch
import math
import torch
class Config(object): 
  def __init__(self):
        self.n_heads = 12
        self.n_layers = 6
        self.pad_token_id = 0
        self.dim = 768####The size of tensor a (768)
        self.hidden_dim = 3072
        self.max_position_embeddings = 512
        self.vocab_size = 30522
        self.eps = 1e-12
        self.attention_head_size = int(self.dim / self.n_heads)
        self.all_head_size = self.n_heads * self.attention_head_size 
        self.n_classes = 2
        self.device = 'cpu'
config = Config()


In [11]:
torch.manual_seed(0) # set seed for reproducible random initialization of weights

class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.dim, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.dim)
        self.LayerNorm = nn.LayerNorm(config.dim, eps=config.eps)

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        """
        Parameters:
            input_ids (torch.Tensor):
                torch.tensor(bs, max_seq_length) The token ids to embed.
        Returns: torch.tensor(bs, max_seq_length, dim) The embedded tokens (plus position embeddings)
        """
 
        # Embedding the input ids
        input_embeds = self.word_embeddings(input_ids)  # (bs, max_seq_length, dim)
        seq_length = input_embeds.size(1)

        # Creating and embedding the position ids  
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)  # (max_seq_length)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)  # (bs, max_seq_length)
        position_embeddings = self.position_embeddings(position_ids)  # (bs, max_seq_length, dim)

        # Compute the output embeddings
        
        # 1. START YOUR CODE HERE #
        embeddings = input_embeds + position_embeddings

        # 1. END YOUR CODE HERE #

        embeddings = self.LayerNorm(embeddings)  # (bs, max_seq_length, dim)
        return embeddings

embedding_layer = Embeddings(config)
# Test if your embedding layer computes an output
embeddings = embedding_layer(inputs['input_ids'])


In [17]:
class AttentionBlock(nn.Module):
  def __init__(self, config): 
    super().__init__() 
    self.config = config

    self.q_lin = nn.Linear(in_features=config.dim, out_features=config.all_head_size)###all_head_size:self.n_heads * self.attention_head_size 
    self.k_lin = nn.Linear(in_features=config.dim, out_features=config.all_head_size)
    self.v_lin = nn.Linear(in_features=config.dim, out_features=config.all_head_size)


    self.out_lin = nn.Linear(in_features=config.dim,out_features=config.dim, bias=True)
    self.sa_layer_norm = nn.LayerNorm(normalized_shape=config.dim,eps=config.eps)
    self.lin1 = nn.Linear(in_features=config.dim,out_features=3072, bias=True)
    self.lin2 = nn.Linear(in_features=3072,out_features=config.dim, bias=True)
    self.output_layer_norm =nn.LayerNorm(normalized_shape=config.dim, eps=config.eps)
  def forward(self, hidden_states):
    def shape(x):
      return x.view(1, -1, 12, 64).transpose(1, 2)
    def unshape(x):
      return x.transpose(1, 2).contiguous().view(1, -1, 12 * 64)

    bs=hidden_states.shape[0]
    n_nodes= hidden_states.shape[1]
    query=key=value=hidden_states
    q = self.q_lin(query)
    k = self.k_lin(key)
    v = self.v_lin(value)
    q = shape(q) # (bs, n_heads, q_length, dim_per_head) k = shape(k) # (bs, n_heads, k_length, dim_per_head) v = shape(v) # (bs, n_heads, k_length, dim_per_head)
     # Normalizing the query-tensor
    q = q / math.sqrt(q.shape[-1]) # 2. START YOUR CODE HERE #
        # Compute attention scores
        #dot product q with all key 
    weights = torch.zeros_like(q)
    for i in range (q.size(1)):
           dot_product = torch.matmul(q[:, i, :], k.transpose(-2, -1))
           #scale
           scaled_dot_product = dot_product / torch.sqrt(k.size(-1))
           attention_weights = torch.softmax(scaled_dot_product, dim=-1)

           weighted_sum = torch.sum(v.unsqueeze(1) * attention_weights.unsqueeze(-1), dim=-2)

           # Assign output for current query
           weights[:, i, :] = weighted_sum.squeeze(1)
    context = unshape(context) # (bs, q_length, dim)
    sa_output = self.out_lin(context)
    sa_output = self.sa_layer_norm(sa_output + hidden_states)
    x = self.lin1(sa_output)
    x = nn.functional.gelu(x)
    ffn_output = self.lin2(x)
    ffn_output = self.output_layer_norm(ffn_output + sa_output)
    return ffn_output, weights
    
block = AttentionBlock(config)
# Test if your attention block computes an output block_output = block(embeddings)





In [ ]:
#task3

torch.manual_seed(0)
class DistillBertAttention(nn.Module): 
  def __init__(self, config):
        super().__init__()
        self.config = config
        self.n_layers=config.n_layers
        self.embeddings = Embeddings(config)
        #encoder
        self.attention_layers = torch.nn.Sequential(*[
            AttentionBlock(config) for _ in range(config.n_layers)
        ])
        # classification
        self.pre_classifier =  nn.Linear(in_features=config.dim,out_features=config.dim, bias=True)#######get the input data and go into hidden_state
        self.classifier =  nn.Linear(in_features=config.dim,out_features=config.n_classes, bias=True)
        self.attention_probs = {i: [] for i in range(config.n_layers)}
  def forward(self, input_ids):
        # Computing the embeddings
        hidden_states =self.embeddings(input_ids=input_ids).to(self.config.device)
        # Iteratively going through the attention layers
        encoder_input = hidden_states 

        for idx, layer in enumerate(self.attention_layers):
            hidden_states, w = layer(hidden_states)
            self.attention_probs[idx] = w###to keep track which attention layer add
        pooled_output = output[:, 0] # (bs, dim)
        
        out = self.pre_classifier(pooled_output)#####behind-nn.Linear
        out = torch.nn.functional.relu(out)
        logits = self.classifier(out)
        
        
        return logits



model = DistillBertAttention(config)
state_dict = torch.load('distilbert.pt')
_ = model.load_state_dict(state_dict)
_ = model.eval()
# Predict your output
logits = model(inputs['input_ids'])


In [20]:
#1. Use the tokenizer to map the 'input_ids' back to 'tokens'.
#2. Extract attention probabilities for every layer by averaging over the attention heads
#in each layer. You should get a matrix of size [n_layers x seq_length x seq_length]
#3. For each layer plot the resulting attention matrix. Hint: Use cmap='Reds' and
#vmin=0, vmax=1.

import numpy as np
import matplotlib.pyplot as plt


tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])#The tokenizer takes the input sentence and converts it into a sequence of tokens that can be fed into a neural network. 
cofig=config()

attention_layers = []
for each_layer in range(config.n_layers):
    layer_attention = torch.mean(model.attention_probs[each_layer], dim=1)  # average over attention heads
    layer_attention = layer_attention.view(1, layer_attention.size(0), layer_attention.size(1))  # add batch dimension
    attention_layers.append(layer_attention)


##plot

for each_layer in range(config.n_layers):
    fig, ax = plt.subplots()
    ax.set_title(f"Layer: {each_layer}")
    ax.imshow(attention_layers[each_layer], cmap="Reds", vmin=0, vmax=1)
    ax.set_xticklabels(tokens, rotation=90)
    ax.set_yticklabels(tokens)
    ax.xaxis.tick_top()
    plt.show()





